<a href="https://colab.research.google.com/github/nagasatvika/semantic-similarity/blob/main/Word_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Importing pandas library
2. Using "as" keyword to create alias pd




In [1]:
import pandas as pd

1. Uploading and mounting the file into google drive  
2. Copy path and putting it in read_csv function of pandas for reading the file to DataFrame
3. Using a delimiter tab as the words are seperated by tab spaces

In [2]:
df = pd.read_csv("/content/drive/MyDrive/nlp/word.txt",delimiter="\t")

The DataFrame df contains 10 different columns as follows

In [3]:
df

,word1,word2,POS,SimLex999,conc(w1),conc(w2),concQ,Assoc(USF),SimAssoc333,SD(SimLex)
0,old,new,A,1.58,2.72,2.81,2,7.25,1,0.41
1,smart,intelligent,A,9.20,1.75,2.46,1,7.11,1,0.67
2,hard,difficult,A,8.77,3.76,2.21,2,5.94,1,1.19
3,happy,cheerful,A,9.55,2.56,2.34,1,5.85,1,2.18
4,hard,easy,A,0.95,3.76,2.07,2,5.82,1,0.93
...,...,...,...,...,...,...,...,...,...,...
994,join,acquire,V,2.85,2.86,2.93,2,0.00,0,0.99
995,send,attend,V,1.67,2.70,3.17,2,0.00,0,1.44
996,gather,attend,V,4.80,2.75,3.17,2,0.00,0,1.97
997,absorb,withdraw,V,2.97,3.11,3.04,2,0.00,0,1.75


#Constrained Setting

#Using SimLex999
Resources Referred
 [pandas](https://www.geeksforgeeks.org/different-ways-to-iterate-over-rows-in-pandas-dataframe/)

Simlex999 is a gold standard resource for evaluating distributional semantic models

In this code, used the SimLex999 column to find out the similarity based on following conditions

1. if 0 <= row['SimLex999'] < 3 then low similar
2. if 3 <= row['SimLex999'] < 6 then moderately similar
3. if 6 <= row['SimLex999'] < 9 then very similar
4. if 9 <= row['SimLex999'] <= 10 then highly similar



In [4]:

def check_sim(row):
  if row['SimLex999']>=0 and row['SimLex999']<3:
    return "low similar"
  elif row['SimLex999']>=3 and row['SimLex999']<6:
    return "moderately similar"
  elif row['SimLex999']>=6 and row['SimLex999']<9:
    return "very similar"
  elif row['SimLex999']>=9 and row['SimLex999']<=10:
    return "highly similar"
df['similarity'] = df.apply(check_sim,axis=1)
for ind in df.index:
  print(f"{df['word1'][ind]}\t{df['word2'][ind]}\t{df['similarity'][ind]}")

old	new	low similar
smart	intelligent	highly similar
hard	difficult	very similar
happy	cheerful	highly similar
hard	easy	low similar
fast	rapid	very similar
happy	glad	highly similar
short	long	low similar
stupid	dumb	highly similar
weird	strange	very similar
wide	narrow	low similar
bad	awful	very similar
easy	difficult	low similar
bad	terrible	very similar
hard	simple	low similar
smart	dumb	low similar
insane	crazy	highly similar
happy	mad	low similar
large	huge	highly similar
hard	tough	very similar
new	fresh	very similar
sharp	dull	low similar
quick	rapid	highly similar
dumb	foolish	very similar
wonderful	terrific	very similar
strange	odd	highly similar
happy	angry	low similar
narrow	broad	low similar
simple	easy	highly similar
old	fresh	low similar
apparent	obvious	very similar
inexpensive	cheap	very similar
nice	generous	moderately similar
weird	normal	low similar
weird	odd	highly similar
bad	immoral	very similar
sad	funny	low similar
wonderful	great	very similar
guilty	ashamed	ve

PROS:        
1.Simlex999 is derived from the human judgements giving similarity score accuracy from moderate to highest

2.As it's already given in the dataset it was easy to implement using pandas

CONS:
1. It is only limited to only on one attribute i.e Simlex999



#Constrained Setting

#Wordnet using :
1. Wu palmer Similarity
2. path Similarity
1. Leacock Chordorow (LCH) Similarity







WORDNET:
Wordnet is large Lexical Database

Resource Referred: [link text](https://medium.com/@ompramod9921/wordnet-in-action-enhancing-nlp-capabilities-through-nltk-2aa922a57d43)()

1.   Importing NTLK Library
2.   downloading wordnet
3.   importing wordnet from nltk

1.  create alias "wn" using "as" keyword





In [5]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn

[nltk_data] Downloading package wordnet to /root/nltk_data...




1. Extracting the first 20 rows from 'word1' and 'word2' of DataFrame df
1.   Define a function 'first_synset' which returns first synset of a given word(assuming that every word has atleast one synset) if that fails we return None
2.   Initialize an empty list named 'similarities' to store the result
1.   Using a for loop to iterate over the first 20 words then

 >Apply 'first_synset' function to the 'word1' and 'word2' to get their first synset

  >Calculating Wu-palmer similarity score between both synsets of 'word1' and 'word2'

 >Appending a tuple 'word1','word2' and 'similarity' to the list 'similarities'
1.   Creating a Dataframe of 'similarities' list using pandas library alias 'pd' named 'result'
2.   Print 'result' DataFrame










In [6]:
w1 = df['word1'][:20]
w2 = df['word2'][:20]

def first_synset(word):
    synsets = wn.synsets(word)
    return synsets[0]

similarities = []

for word1, word2 in zip(w1, w2):
    syn1 = first_synset(word1)
    syn2 = first_synset(word2)
    similarity1 = syn1.wup_similarity(syn2)
    similarity2 = syn1.path_similarity(syn2)
    similarity3 = syn1.lch_similarity(syn2) if syn1.pos()==syn2.pos() else 0
    similarities.append((word1,word2,similarity1,similarity2,similarity3))

results = pd.DataFrame(similarities,columns=['word1','word2',similarity1,similarity2,similarity3])
print(results)



     word1        word2       0.2  0.1111111111111111         0
0      old          new  0.222222            0.125000  0.000000
1    smart  intelligent  0.166667            0.090909  0.000000
2     hard    difficult  1.000000            1.000000  0.693147
3    happy     cheerful  0.500000            0.333333 -0.405465
4     hard         easy  0.500000            0.333333 -0.405465
5     fast        rapid  0.125000            0.066667  0.929536
6    happy         glad  0.142857            0.076923  0.000000
7    short         long  0.166667            0.090909  0.000000
8   stupid         dumb  0.200000            0.111111  0.000000
9    weird      strange  0.153846            0.083333  0.000000
10    wide       narrow  0.200000            0.111111  0.000000
11     bad        awful  0.250000            0.142857  0.000000
12    easy    difficult  0.500000            0.333333 -0.405465
13     bad     terrible  0.250000            0.142857  0.000000
14    hard       simple  0.153846       

Euclidean Distance

#Hamming Distance


In [ ]:
from scipy.spatial import distance
import numpy as np
w1 = df['word1'][:20]
w2 = df['word2'][:20]
similarities = []
for word1,word2 in zip(w1,w2):
  max_len = max(len(word1),len(word2))
  word1 = word1.ljust(max_len)
  word2 = word2.ljust(max_len)
  Normalized_HD = distance.hamming(list(word1),list(word2))
  hamming = Normalized_HD*max_len
  similarities.append((word1,word2,hamming))
result = pd.DataFrame(similarities,columns=['word1','word2','hamming'])
print(result)


          word1        word2  hamming
0           old          new      3.0
1   smart        intelligent     11.0
2     hard         difficult      9.0
3      happy        cheerful      8.0
4          hard         easy      3.0
5         fast         rapid      4.0
6         happy        glad       5.0
7         short        long       5.0
8        stupid       dumb        6.0
9       weird        strange      7.0
10       wide         narrow      6.0
11        bad          awful      5.0
12    easy         difficult      9.0
13     bad          terrible      8.0
14       hard         simple      6.0
15        smart        dumb       5.0
16       insane       crazy       6.0
17        happy        mad        4.0
18        large        huge       5.0
19        hard         tough      5.0


#Unconstrained Setting

#Spacy Similarity method
Resources referred :

1.  [tutorial](https://campus.datacamp.com/courses/advanced-nlp-with-spacy/large-scale-data-analysis-with-spacy?ex=8)
2.  [paper](https://www.irjmets.com/uploadedfiles/paper/issue_9_september_2022/30252/final/fin_irjmets1664350077.pdf)





Using pip to install spacy library





In [7]:
!pip install spacy



1.   Using 'python -m spacy' which means to run spacy module
2.   Downloading en_core_web_md which is Medium-sized English model



In [8]:
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 20.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


1.   Importing spacy library
2.   loading en_core_web_md i.e medium sized english pre-trained model



In [9]:
import spacy
nlp = spacy.load('en_core_web_md')

1. Extracting the first 10 rows from 'word1' and 'word2' of DataFrame df
2. Initialize an empty list named 'similarities' to store the result
1. Using a for loop to iterate over the first 20 words then
> applying nlp for word1 and word2 which creates a spacy document

 >  we use similarity method to find the similarity scores of word1 and word2

 > Appending a tuple ('word1','word2','similarity') to the list 'similarities'



4.   Creating a Dataframe of 'similarities' list using pandas library alias 'pd' named 'result'
2.   Print 'result' DataFrame



In [10]:
w1 = df['word1'][:10]
w2 = df['word2'][:10]

similarities = []

for word1,word2 in zip(w1,w2):
  W1 = nlp(word1)
  W2 = nlp(word2)
  Similarity = W1.similarity(W2)
  similarities.append((word1,word2,Similarity))
result = pd.DataFrame(similarities,columns=['word1','word2','Similarity'])
print(result)

    word1        word2  Similarity
0     old          new    0.157152
1   smart  intelligent    0.592976
2    hard    difficult    0.698994
3   happy     cheerful    0.606921
4    hard         easy    0.554700
5    fast        rapid    0.596927
6   happy         glad    0.618630
7   short         long    0.678049
8  stupid         dumb    0.840375
9   weird      strange    0.672306


#Using Sentence Transformers
Resource Referred :

1.  [sbert](https://www.sbert.net/docs/sentence_transformer/usage/semantic_textual_similarity.html)







1.  Installing **sentence-transformers**
2.  **-q** is a flag used to supress the output during installation process



In [11]:
%pip install sentence-transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 67.3 MB/s eta 0:00:00


1. Importing **SentenceTransformer** CLASS from **sentence_transformers**
2. util is a module which includes functions such as cosine similarity
1.  The **SentenceTransformer** Class is *instantiated* with **all-MiniLM-L6-v2**
 which generates the *vector* *representations* of given *sentence*





In [12]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]



1.  Extracting the first 20 rows from 'word1' and 'word2' of DataFrame df
2.  Generating vector representations/embeddings of words w1,w2 using SentenceTransformer model
1. Then Calculating the cosine similarity between the vector representations/embeddings
2. Using a for for iterating through the first 20 words and printing similarity scores





In [13]:
w1 = df['word1'][:20]
w2 = df['word2'][:20]
embeddings1 = model.encode(w1, convert_to_tensor=True)
embeddings2 = model.encode(w2, convert_to_tensor=True)
cosine_score = util.cos_sim(embeddings1,embeddings2)
for i in range(20):
  cosine_score_value = cosine_score[i][i].item()
  print("{} \t\t {} \t\t Score:{:.4f}".format(w1[i], w2[i],cosine_score_value))

old 		 new 		 Score:0.5034
smart 		 intelligent 		 Score:0.7494
hard 		 difficult 		 Score:0.8246
happy 		 cheerful 		 Score:0.5541
hard 		 easy 		 Score:0.7239
fast 		 rapid 		 Score:0.7319
happy 		 glad 		 Score:0.5697
short 		 long 		 Score:0.6555
stupid 		 dumb 		 Score:0.7669
weird 		 strange 		 Score:0.8870
wide 		 narrow 		 Score:0.8235
bad 		 awful 		 Score:0.6512
easy 		 difficult 		 Score:0.7745
bad 		 terrible 		 Score:0.7928
hard 		 simple 		 Score:0.5171
smart 		 dumb 		 Score:0.6009
insane 		 crazy 		 Score:0.8316
happy 		 mad 		 Score:0.3313
large 		 huge 		 Score:0.8102
hard 		 tough 		 Score:0.7784
